In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.tensor([
    [0.72, 0.45, 0.31],
    [0.75, 0.20, 0.55],
    [0.30, 0.80, 0.40],
    [0.85, 0.35, 0.60],
    [0.55, 0.15, 0.75],
    [0.25, 0.20, 0.85]
])

In [3]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [4]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, _ = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens],
            -torch.inf
        )

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5,
            dim=-1
        )

        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec


In [5]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [
                CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
                for _ in range(num_heads)
            ]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


In [6]:
torch.manual_seed(123)

context_length = batch.shape[1]
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(
    d_in=d_in,
    d_out=d_out,
    context_length=context_length,
    dropout=0.0,
    num_heads=2
)

In [7]:
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.5762, -0.1627,  0.5569,  0.3635],
         [-0.5650, -0.0630,  0.5599,  0.3006],
         [-0.5472, -0.1226,  0.5285,  0.3435],
         [-0.5787, -0.0943,  0.5621,  0.3388],
         [-0.5593, -0.0436,  0.5509,  0.3046],
         [-0.5287, -0.0033,  0.5277,  0.2743]],

        [[-0.5762, -0.1627,  0.5569,  0.3635],
         [-0.5650, -0.0630,  0.5599,  0.3006],
         [-0.5472, -0.1226,  0.5285,  0.3435],
         [-0.5787, -0.0943,  0.5621,  0.3388],
         [-0.5593, -0.0436,  0.5509,  0.3046],
         [-0.5287, -0.0033,  0.5277,  0.2743]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [8]:
# Input: (batch=1, seq_len=3, d_model=6)
x = torch.tensor([[
    [1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
    [6.0, 5.0, 4.0, 3.0, 2.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
]])

batch_size, seq_len, d_model = x.shape


In [9]:
torch.manual_seed(0)

Wq = torch.randn(d_model, d_model)
Wk = torch.randn(d_model, d_model)
Wv = torch.randn(d_model, d_model)


In [10]:
Q = x @ Wq
K = x @ Wk
V = x @ Wv

In [11]:
num_heads = 2
head_dim = d_model // num_heads

In [12]:
Q = Q.view(batch_size, seq_len, num_heads, head_dim)
K = K.view(batch_size, seq_len, num_heads, head_dim)
V = V.view(batch_size, seq_len, num_heads, head_dim)

In [13]:
Q = Q.transpose(1, 2)
K = K.transpose(1, 2)
V = V.transpose(1, 2)

In [14]:
K_T = K.transpose(2, 3)
attn_scores = Q @ K_T

In [15]:
mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
attn_scores.masked_fill_(mask, -torch.inf)

tensor([[[[-318.2692,      -inf,      -inf],
          [-294.6535,    9.5538,      -inf],
          [ -87.5604,   -1.7744,  -12.7621]],

         [[ 116.1476,      -inf,      -inf],
          [ 178.4425,  171.2106,      -inf],
          [  42.0843,   31.7945,   10.5541]]]])

In [16]:
attn_weights = torch.softmax(attn_scores / head_dim**0.5, dim=-1)

In [17]:
dropout = nn.Dropout(0.1)
attn_weights = dropout(attn_weights)

In [18]:
context_vec = attn_weights @ V

In [19]:
context_vec = context_vec.transpose(1, 2)

In [20]:
context_vec = context_vec.reshape(batch_size, seq_len, num_heads * head_dim)

In [21]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)

        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, _ = x.shape

        Q = self.W_query(x)
        K = self.W_key(x)
        V = self.W_value(x)

        Q = Q.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)

        attn_scores = Q @ K.transpose(2, 3)
        attn_scores.masked_fill_(
            self.mask[:num_tokens, :num_tokens].bool(),
            -torch.inf
        )

        attn_weights = torch.softmax(
            attn_scores / self.head_dim**0.5,
            dim=-1
        )

        attn_weights = self.dropout(attn_weights)
        context = attn_weights @ V

        context = context.transpose(1, 2).reshape(b, num_tokens, self.d_out)
        output = self.out_proj(context)

        return output


In [22]:
torch.manual_seed(123)

mha = MultiHeadAttention(
    d_in=6,
    d_out=6,
    context_length=seq_len,
    dropout=0.1,
    num_heads=2
)

out = mha(x)
print(out)
print(out.shape)


tensor([[[ 0.1556,  0.0844,  1.1247,  0.6199, -1.1666, -0.0814],
         [ 0.1454,  0.0948,  1.1142,  0.6164, -1.1659, -0.0827],
         [-0.1063,  0.3392,  0.3488,  0.2884, -0.7438, -0.1504]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 3, 6])


## 🔷 Multi-Head Attention (Mapped Directly to Code)

**Initialize dimensions** by enforcing `d_out % num_heads == 0` so that *each attention head has equal capacity*.  
Compute the per-head dimension as **`head_dim = d_out // num_heads`**.

---

**Define learnable linear projections**  
The layers **`W_query`**, **`W_key`**, and **`W_value`** project the input embeddings into:
- **Query (Q)**
- **Key (K)**
- **Value (V)**  

Each projection produces tensors of shape **`(b, num_tokens, d_out)`**.

---

**Register a causal mask**  
A **causal upper-triangular mask** is registered using `register_buffer`.  
This mask:
- Is **stored with the model**
- Is **not trainable**
- Prevents tokens from attending to **future positions**

---

**Forward pass begins**  
The input tensor **`x`** has shape **`(b, num_tokens, d_in)`**.  
Compute the projections:
- `Q = W_query(x)`
- `K = W_key(x)`
- `V = W_value(x)`

---

**Split into multiple heads (reshape)**  
Each projection is reshaped to:

**`(b, num_tokens, num_heads, head_dim)`**

This step:
- Splits the embedding dimension
- Assigns a smaller subspace to each head
- *Does not change the underlying values*

---

**Group by head (transpose)**  
Transpose each tensor to:

**`(b, num_heads, num_tokens, head_dim)`**

This reordering allows **attention to be computed independently for each head**.

---

**Scaled dot-product attention**  
Compute attention scores using:

`Q @ K.transpose(2, 3)`

This produces scores of shape **`(b, num_heads, num_tokens, num_tokens)`**.  
Apply the **causal mask** to block future tokens, then apply **softmax scaled by `sqrt(head_dim)`** to obtain normalized attention weights.

---

**Dropout and context computation**  
Apply **dropout** to the attention weights for regularization.  
Compute the context vectors using:

`attn_weights @ V`

Resulting shape: **`(b, num_heads, num_tokens, head_dim)`**

---

**Merge heads back together**  
Transpose the context back to:

**`(b, num_tokens, num_heads, head_dim)`**

Then reshape to:

**`(b, num_tokens, d_out)`**

This concatenates all heads into a single representation.

---

**Final output projection**  
Apply the output projection **`out_proj`** to mix information from all heads and produce the final output:

**`(b, num_tokens, d_out)`**
